In [33]:
import pandas as pd
import numpy as np
from PIL import Image
#from wordcloud import WordCloud
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [40]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wangd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wangd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wangd\AppData\Roaming\nltk_data...


True

In [41]:
df=pd.read_csv('spam.csv', encoding='ISO-8859-1')
df=df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [43]:
def basic_clean(string):
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    string = BeautifulSoup(string, "html.parser").get_text()
    string = re.sub(r'\S+@\S+', '', string)
    string = re.sub(r'http\S+|www\S+|https\S+', '', string, flags=re.MULTILINE)
    string = re.sub(r'\d+', '', string)
    string = re.sub(r'[^\w\s]', '', string).lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in string.split()]
    string = ' '.join(lemmatized_words)
    return string

In [44]:
def remove_stopwords(string, extra_words=[], exclude_words=[]):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # Create stopword_list.
    stopword_list = stopwords.words('english')

    #remove 'exclude_words' from stopword_list
    stopword_list=set(stopword_list)-set(exclude_words)
    
    # Add in 'extra_words' to stopword_list.
    stopword_list = stopword_list.union(set(extra_words))

    # Split words in string.
    words = string.split()
    
    # Create a list of words from my string with stopwords removed and assign to variable.
    filtered_words = [word for word in words if word not in stopword_list]
    
    # Join words in the list back into strings and assign to a variable.
    string_without_stopwords = ' '.join(filtered_words)
    
    return string_without_stopwords

    

In [45]:
df['no_stopwords']=df['v2'].apply(basic_clean).apply(remove_stopwords)
df['tokens'] = df['no_stopwords'].apply(word_tokenize)

C:\Users\wangd\AppData\Local\Temp\ipykernel_74748\3129113494.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  string = BeautifulSoup(string, "html.parser").get_text()


In [ ]:
from collections import Counter

ham_df = df[df['v1'] == 'ham']
spam_df = df[df['v1'] == 'spam']

# Flatten the list of tokens for each class
ham_tokens = [token for tokens in ham_df['tokens'] for token in tokens]
spam_tokens = [token for tokens in spam_df['tokens'] for token in tokens]

# Count the frequency of each word for each class
ham_word_freq = Counter(ham_tokens)
spam_word_freq = Counter(spam_tokens)

# Display the most common words for each class
print("Most common words in 'ham':")
print(ham_word_freq.most_common())

print("\nMost common words in 'spam':")
print(spam_word_freq.most_common())

Most common words in 'ham':
[('u', 972), ('im', 449), ('2', 305), ('get', 303), ('ltgt', 276), ('ok', 272), ('dont', 257), ('go', 247), ('got', 243), ('ur', 240), ('ill', 236), ('know', 232), ('like', 229), ('call', 229), ('come', 224), ('good', 222), ('time', 189), ('day', 187), ('love', 185), ('4', 168), ('going', 167), ('want', 163), ('one', 162), ('home', 160), ('lor', 160), ('need', 156), ('sorry', 153), ('still', 146), ('see', 137), ('n', 134), ('later', 134), ('da', 131), ('r', 131), ('back', 129), ('think', 128), ('well', 126), ('today', 125), ('send', 123), ('tell', 121), ('cant', 118), ('i_', 117), ('hi', 117), ('take', 112), ('much', 112), ('oh', 111), ('night', 107), ('hey', 106), ('happy', 105), ('great', 100), ('way', 100), ('hope', 99), ('pls', 98), ('na', 96), ('work', 96), ('wat', 95), ('thats', 94), ('dear', 94), ('give', 92), ('say', 91), ('already', 89), ('right', 89), ('make', 88), ('ask', 88), ('said', 86), ('really', 85), ('yeah', 85), ('amp', 82), ('wan', 81), (

In [ ]:
def compute_word_frequency(df):
    vectorizer = CountVectorizer(stop_words='english')

    # Fit and transform the cleaned sentences
    X = vectorizer.fit_transform(df['no_stopwords'])

    # Sum the counts of each word
    word_counts = X.toarray().sum(axis=0)

    # Get the words
    words = vectorizer.get_feature_names_out()

    # Create a DataFrame with words and their counts
    word_freq_df = pd.DataFrame({'word': words, 'count': word_counts})

    # Sort the DataFrame by count in descending order
    word_freq_df = word_freq_df.sort_values(by='count', ascending=False).head(10)
    return word_freq_df


In [ ]:
compute_word_frequency(ham_df)

,word,count
3103,im,449
3787,ltgt,276
4446,ok,272
1881,dont,257
6699,ur,240
3100,ill,236
3488,know,232
2648,got,231
3649,like,229
1351,come,224


In [ ]:
compute_word_frequency(spam_df)

,word,count
1360,free,216
2564,txt,150
2603,ur,144
1805,mobile,123
2473,text,120
2382,stop,113
1067,claim,113
2169,reply,101
2072,prize,92
1864,new,69


In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words='english')

# Vectorize and compute bigram frequencies for each class
def compute_ngram_frequencies(df, vectorizer):
    X = vectorizer.fit_transform(df['no_stopwords'])
    ngram_counts = X.toarray().sum(axis=0)
    ngrams = vectorizer.get_feature_names_out()
    ngram_freq_df = pd.DataFrame({'ngram': ngrams, 'count': ngram_counts})
    ngram_freq_df = ngram_freq_df.sort_values(by='count', ascending=False).head(10)
    return ngram_freq_df

In [ ]:
compute_ngram_frequencies(ham_df, bigram_vectorizer)

,ngram,count
9166,ill later,43
10811,let know,40
17848,sorry ill,39
4780,dont know,32
7112,good morning,30
9306,im going,24
13508,new year,23
15061,pls send,22
14052,ok lor,22
9309,im gonna,20


In [ ]:
compute_ngram_frequencies(spam_df, bigram_vectorizer)

,ngram,count
3713,po box,24
841,a1000 cash,23
3853,prize guaranteed,22
4239,send stop,20
4209,selected receive,19
1100,await collection,19
2828,land line,18
5155,urgent mobile,18
1415,chance win,17
5186,valid 12hrs,17


In [ ]:
compute_ngram_frequencies(spam_df, CountVectorizer(ngram_range=(3, 3), stop_words='english'))

,ngram,count
2907,land line claim,14
399,2003 account statement,13
3986,private 2003 account,13
4542,shows 800 unredeemed,12
2575,guaranteed a1000 cash,12
5458,urgent trying contact,11
1323,camcorder reply 08000930705,11
1210,bonus caller prize,11
1362,cash await collection,10
841,a2000 bonus caller,10


In [ ]:
compute_ngram_frequencies(ham_df, CountVectorizer(ngram_range=(3, 3), stop_words='english'))

,ngram,count
16672,sorry ill later,38
7222,happy new year,18
14157,pls send message,13
15147,right pls send,12
13885,phone right pls,12
13938,pick phone right,12
7687,hi hi hi,11
6471,good morning dear,7
3059,convey birthday wishes,6
8519,ill later meeting,6


In [ ]:
#with preprocessing
vectorizer = CountVectorizer(stop_words='english')

y=df['v1']
X=df['no_stopwords']

X_train, X_test, y_train, y_test = train_test_split(X, df['v1'], test_size=0.2, random_state=123)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       982
        spam       1.00      0.79      0.88       133

    accuracy                           0.97      1115
   macro avg       0.99      0.89      0.93      1115
weighted avg       0.98      0.97      0.97      1115



In [ ]:
#with preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

X_train, X_test, y_train, y_test = train_test_split(X, df['v1'], test_size=0.2, random_state=123)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       982
        spam       1.00      0.77      0.87       133

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [47]:
#without preprocessing
vectorizer = TfidfVectorizer(stop_words='english')

X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'], test_size=0.2, random_state=123)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       982
        spam       0.99      0.78      0.87       133

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [48]:
#without preprocessing
vectorizer = CountVectorizer(stop_words='english')
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'], test_size=0.2, random_state=123)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       982
        spam       1.00      0.80      0.89       133

    accuracy                           0.98      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.98      0.98      0.97      1115

